# 📊 Análisis Exploratorio de Datos (EDA) - PAKDD 2010 Credit Risk

Este notebook contiene el análisis exploratorio del dataset PAKDD 2010 para predicción de riesgo crediticio.

## 🎯 Objetivos:
- Cargar y explorar los datos
- Entender la estructura del dataset
- Analizar distribuciones y patrones
- Identificar problemas de calidad de datos
- Preparar insights para feature engineering

## 📚 Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Librerías importadas correctamente")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

## 📁 Cargar los Datos

Vamos a cargar los datos de PAKDD 2010. Este dataset proviene de una competencia real de data mining.

In [ ]:
# Intentar cargar los datos con diferentes separadores
try:
    # Primero intentemos con tabulaciones
    df_train = pd.read_csv('data/raw/PAKDD2010_Modeling_Data.txt', sep='\t', low_memory=False)
    print(f"✅ Datos cargados con separador TAB: {df_train.shape}")
except:
    try:
        # Si no funciona, intentemos con espacios múltiples
        df_train = pd.read_csv('data/raw/PAKDD2010_Modeling_Data.txt', sep='\s+', low_memory=False)
        print(f"✅ Datos cargados con separador de espacios: {df_train.shape}")
    except Exception as e:
        print(f"❌ Error cargando datos: {e}")
        # Intentemos leer las primeras líneas para ver el formato
        with open('data/raw/PAKDD2010_Modeling_Data.txt', 'r') as f:
            lines = f.readlines()[:3]
            print("Primeras líneas del archivo:")
            for i, line in enumerate(lines, 1):
                print(f"Línea {i}: {line[:100]}...")

## 🔍 Exploración Inicial de los Datos

In [ ]:
# Información básica del dataset
print("📊 INFORMACIÓN BÁSICA DEL DATASET")
print("=" * 50)
print(f"📐 Dimensiones: {df_train.shape[0]:,} filas x {df_train.shape[1]} columnas")
print(f"💾 Tamaño en memoria: {df_train.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"🔢 Tipos de datos:")
print(df_train.dtypes.value_counts())

In [ ]:
# Ver las primeras filas
print("👀 PRIMERAS 5 FILAS:")
print("=" * 30)
display(df_train.head())

In [ ]:
# Ver las últimas filas
print("👀 ÚLTIMAS 5 FILAS:")
print("=" * 30)
display(df_train.tail())

In [ ]:
# Información detallada
print("📋 INFORMACIÓN DETALLADA:")
print("=" * 35)
df_train.info()

## 🎯 Identificar Variable Objetivo

En datasets de riesgo crediticio, la variable objetivo suele ser:
- `target`, `default`, `bad_loan`, etc.
- Típicamente la última columna
- Valores binarios (0/1, Y/N, etc.)

In [ ]:
# Analizar la última columna (probable variable objetivo)
last_col = df_train.columns[-1]
print(f"🎯 ANÁLISIS DE LA ÚLTIMA COLUMNA: '{last_col}'")
print("=" * 50)
print(f"Valores únicos: {df_train[last_col].unique()}")
print(f"Conteo de valores:")
print(df_train[last_col].value_counts())
print(f"\nPorcentaje:")
print(df_train[last_col].value_counts(normalize=True) * 100)

In [ ]:
# Buscar posibles variables objetivo en todas las columnas
print("🔍 BUSCANDO VARIABLES BINARIAS (POSIBLES TARGETS):")
print("=" * 55)

binary_columns = []
for col in df_train.columns:
    unique_vals = df_train[col].dropna().unique()
    if len(unique_vals) == 2:
        binary_columns.append(col)
        print(f"✓ {col}: {unique_vals}")

print(f"\n📊 Total de columnas binarias encontradas: {len(binary_columns)}")

## 🧮 Estadísticas Descriptivas

In [ ]:
# Estadísticas para variables numéricas
numeric_cols = df_train.select_dtypes(include=[np.number]).columns
print(f"📊 ESTADÍSTICAS DE VARIABLES NUMÉRICAS ({len(numeric_cols)} columnas):")
print("=" * 65)

if len(numeric_cols) > 0:
    display(df_train[numeric_cols].describe())
else:
    print("❌ No se encontraron columnas numéricas")

In [ ]:
# Estadísticas para variables categóricas
categorical_cols = df_train.select_dtypes(include=['object']).columns
print(f"📊 ESTADÍSTICAS DE VARIABLES CATEGÓRICAS ({len(categorical_cols)} columnas):")
print("=" * 70)

for col in categorical_cols[:10]:  # Solo las primeras 10 para no saturar
    unique_count = df_train[col].nunique()
    print(f"\n🏷️  {col}:")
    print(f"   Valores únicos: {unique_count}")
    if unique_count <= 10:
        print(f"   Top valores: {df_train[col].value_counts().head().to_dict()}")
    else:
        print(f"   Top 5 valores: {df_train[col].value_counts().head().to_dict()}")

## 🕳️ Análisis de Valores Faltantes

In [ ]:
# Calcular valores faltantes
missing_data = df_train.isnull().sum()
missing_percent = (missing_data / len(df_train)) * 100

missing_df = pd.DataFrame({
    'Column': df_train.columns,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values
})

# Filtrar solo columnas con valores faltantes
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print(f"🕳️  ANÁLISIS DE VALORES FALTANTES:")
print("=" * 40)
print(f"📊 Columnas con valores faltantes: {len(missing_df)}/{len(df_train.columns)}")

if len(missing_df) > 0:
    print(f"\nTop 10 columnas con más valores faltantes:")
    display(missing_df.head(10))
else:
    print("✅ ¡No hay valores faltantes en el dataset!")

In [ ]:
# Visualización de valores faltantes
if len(missing_df) > 0:
    fig = px.bar(
        missing_df.head(15), 
        x='Missing_Percentage', 
        y='Column',
        title='📊 Top 15 Columnas con Valores Faltantes',
        labels={'Missing_Percentage': 'Porcentaje Faltante (%)', 'Column': 'Columna'},
        color='Missing_Percentage',
        color_continuous_scale='Reds'
    )
    fig.update_layout(height=600)
    fig.show()
else:
    print("✅ Gráfico no necesario - No hay valores faltantes")

## 📈 Visualizaciones Exploratorias

In [ ]:
# Distribución de la variable objetivo (si la identificamos)
if len(binary_columns) > 0:
    target_col = binary_columns[-1]  # Asumimos que la última binaria es el target
    
    fig = px.pie(
        values=df_train[target_col].value_counts().values,
        names=df_train[target_col].value_counts().index,
        title=f'📊 Distribución de la Variable Objetivo: {target_col}'
    )
    fig.show()
    
    print(f"\n🎯 BALANCE DE CLASES en '{target_col}':")
    print("=" * 40)
    class_counts = df_train[target_col].value_counts()
    for class_val, count in class_counts.items():
        percentage = (count / len(df_train)) * 100
        print(f"   {class_val}: {count:,} ({percentage:.1f}%)")
    
    # Calcular ratio de desbalance
    minority_class = class_counts.min()
    majority_class = class_counts.max()
    imbalance_ratio = majority_class / minority_class
    print(f"\n⚖️  Ratio de desbalance: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 10:
        print("⚠️  DATASET MUY DESBALANCEADO - Considerar técnicas de balanceo")
    elif imbalance_ratio > 3:
        print("⚠️  Dataset moderadamente desbalanceado")
    else:
        print("✅ Dataset relativamente balanceado")
else:
    print("❌ No se pudo identificar una variable objetivo clara")

In [ ]:
# Distribuciones de variables numéricas
if len(numeric_cols) > 0:
    # Crear subplots para las primeras 6 variables numéricas
    cols_to_plot = numeric_cols[:6]
    
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=[f'Distribución de {col}' for col in cols_to_plot]
    )
    
    for i, col in enumerate(cols_to_plot):
        row = i // 3 + 1
        col_pos = i % 3 + 1
        
        fig.add_trace(
            go.Histogram(x=df_train[col].dropna(), name=col, showlegend=False),
            row=row, col=col_pos
        )
    
    fig.update_layout(
        title='📊 Distribuciones de Variables Numéricas',
        height=800
    )
    fig.show()
else:
    print("❌ No hay variables numéricas para graficar")

## 🔍 Análisis de Calidad de Datos

In [ ]:
# Análisis de duplicados
duplicates = df_train.duplicated().sum()
duplicate_percentage = (duplicates / len(df_train)) * 100

print(f"🔍 ANÁLISIS DE CALIDAD DE DATOS:")
print("=" * 40)
print(f"📊 Filas duplicadas: {duplicates:,} ({duplicate_percentage:.2f}%)")

if duplicates > 0:
    print("⚠️  Hay filas duplicadas que podrían necesitar atención")
else:
    print("✅ No hay filas duplicadas")

In [ ]:
# Análisis de cardinalidad (valores únicos por columna)
cardinality_data = []
for col in df_train.columns:
    unique_count = df_train[col].nunique()
    unique_percentage = (unique_count / len(df_train)) * 100
    cardinality_data.append({
        'Column': col,
        'Unique_Count': unique_count,
        'Unique_Percentage': unique_percentage,
        'Data_Type': str(df_train[col].dtype)
    })

cardinality_df = pd.DataFrame(cardinality_data)
cardinality_df = cardinality_df.sort_values('Unique_Percentage', ascending=False)

print(f"\n🎲 ANÁLISIS DE CARDINALIDAD:")
print("=" * 35)
print("Top 10 columnas con más valores únicos:")
display(cardinality_df.head(10))

# Identificar columnas con alta cardinalidad (posibles IDs)
high_cardinality = cardinality_df[cardinality_df['Unique_Percentage'] > 95]
if len(high_cardinality) > 0:
    print(f"\n🆔 Posibles columnas ID (alta cardinalidad >95%):")
    for col in high_cardinality['Column']:
        print(f"   • {col}")
    print("💡 Estas columnas probablemente deberían excluirse del modelado")

## 📋 Resumen del Análisis Exploratorio

In [ ]:
print("📋 RESUMEN EJECUTIVO DEL EDA")
print("=" * 50)
print(f"📐 Dimensiones del dataset: {df_train.shape[0]:,} filas × {df_train.shape[1]} columnas")
print(f"💾 Tamaño en memoria: {df_train.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"🔢 Variables numéricas: {len(numeric_cols)}")
print(f"🏷️  Variables categóricas: {len(categorical_cols)}")
print(f"🎯 Variables binarias (posibles targets): {len(binary_columns)}")
print(f"🕳️  Columnas con valores faltantes: {len(missing_df) if len(missing_df) > 0 else 0}")
print(f"🔄 Filas duplicadas: {duplicates:,}")

if len(binary_columns) > 0:
    target_col = binary_columns[-1]
    class_counts = df_train[target_col].value_counts()
    imbalance_ratio = class_counts.max() / class_counts.min()
    print(f"⚖️  Ratio de desbalance: {imbalance_ratio:.2f}:1")

print("\n🎯 PRÓXIMOS PASOS RECOMENDADOS:")
print("=" * 40)
print("1. 🧹 Limpieza de datos y manejo de valores faltantes")
print("2. 🔧 Feature engineering y transformaciones")
print("3. 📊 Análisis de correlaciones")
print("4. 🤖 Preparación para modelado")
print("5. ⚖️  Aplicar técnicas de balanceo si es necesario")